In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

# 设置浏览器驱动
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 初始URL
url = 'https://zhangjiakou.esf.fang.com/house-a011434/'

# 存储所有数据的列表
all_data = []

try:
    # 打开初始页面
    driver.get(url)
    print("开始爬取数据...")
    
    # 循环爬取20页
    for page_num in range(20):
        print(f"正在爬取第 {page_num + 1} 页...")
        
        # 等待页面加载完成
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "shop_list"))
        )
        
        # 查找所有的房源元素 (dl标签)
        house_elements = driver.find_elements(By.CSS_SELECTOR, "dl.clearfix")
        
        # 遍历每个房源元素，提取数据
        for house in house_elements:
            try:
                # 1. 提取面积数据
                try:
                    tel_shop = house.find_element(By.CLASS_NAME, "tel_shop")
                    tel_text = tel_shop.text
                    
                    # 使用正则表达式匹配面积数据 (数字+㎡)
                    area_match = re.search(r'(\d+)㎡', tel_text)
                    if area_match:
                        area = area_match.group(1)  # 提取数字部分
                    else:
                        area = "N/A"
                except NoSuchElementException:
                    area = "N/A"
                
                # 2. 提取总价数据
                try:
                    price_right = house.find_element(By.CLASS_NAME, "price_right")
                    red_element = price_right.find_element(By.CLASS_NAME, "red")
                    price_b = red_element.find_element(By.TAG_NAME, "b")
                    total_price = price_b.text
                except NoSuchElementException:
                    total_price = "N/A"
                
                # 3. 提取单价数据
                try:
                    price_right = house.find_element(By.CLASS_NAME, "price_right")
                    spans = price_right.find_elements(By.TAG_NAME, "span")
                    if len(spans) >= 2:
                        unit_price_text = spans[1].text
                        unit_price_match = re.search(r'(\d+)', unit_price_text)
                        unit_price = unit_price_match.group(1) if unit_price_match else "N/A"
                    else:
                        unit_price = "N/A"
                except NoSuchElementException:
                    unit_price = "N/A"
                
                # 将数据添加到列表
                all_data.append({
                    '面积(㎡)': area,
                    '总价(万)': total_price,
                    '均价(元/㎡)': unit_price
                })
                
            except Exception as e:
                # 如果单个房源解析失败，跳过并继续下一个
                print(f"解析房源时出错: {e}")
                continue
        
        print(f"第 {page_num + 1} 页爬取完成，共找到 {len(house_elements)} 个房源")
        
        # 如果不是最后一页，点击下一页
        if page_num < 19:
            try:
                # 查找下一页按钮
                next_page = driver.find_element(By.CLASS_NAME, "last")
                next_link = next_page.find_element(By.TAG_NAME, "a")
                
                # 点击下一页
                next_link.click()
                
                # 等待页面加载
                time.sleep(2)
                
            except NoSuchElementException:
                print("找不到下一页按钮，爬取结束")
                break
            except Exception as e:
                print(f"点击下一页时出错: {e}")
                break

finally:
    # 关闭浏览器
    driver.quit()
    print("浏览器已关闭")

# 将数据转换为DataFrame
if all_data:
    df = pd.DataFrame(all_data)
    
    # 显示前几行数据
    print("\n爬取到的数据前5行:")
    print(df.head())
    
    # 保存到Excel文件，按照要求的列顺序
    df[['面积(㎡)', '总价(万)', '均价(元/㎡)']].to_excel('hebei_esf_huailai_data.xlsx', index=False)
    print(f"\n数据已保存到 '房价数据.xlsx'，共 {len(df)} 条记录")
    
    # 显示数据统计信息
    print("\n数据统计信息:")
    print(df.describe())
else:
    print("没有爬取到任何数据")

开始爬取数据...
正在爬取第 1 页...
第 1 页爬取完成，共找到 60 个房源
正在爬取第 2 页...
第 2 页爬取完成，共找到 60 个房源
正在爬取第 3 页...
第 3 页爬取完成，共找到 60 个房源
正在爬取第 4 页...
第 4 页爬取完成，共找到 60 个房源
正在爬取第 5 页...
第 5 页爬取完成，共找到 60 个房源
正在爬取第 6 页...
第 6 页爬取完成，共找到 60 个房源
正在爬取第 7 页...
第 7 页爬取完成，共找到 60 个房源
正在爬取第 8 页...
第 8 页爬取完成，共找到 60 个房源
正在爬取第 9 页...
第 9 页爬取完成，共找到 60 个房源
正在爬取第 10 页...
第 10 页爬取完成，共找到 60 个房源
正在爬取第 11 页...
第 11 页爬取完成，共找到 60 个房源
正在爬取第 12 页...
第 12 页爬取完成，共找到 60 个房源
正在爬取第 13 页...
第 13 页爬取完成，共找到 60 个房源
正在爬取第 14 页...
第 14 页爬取完成，共找到 60 个房源
正在爬取第 15 页...
第 15 页爬取完成，共找到 60 个房源
正在爬取第 16 页...
第 16 页爬取完成，共找到 60 个房源
正在爬取第 17 页...
第 17 页爬取完成，共找到 60 个房源
正在爬取第 18 页...
第 18 页爬取完成，共找到 60 个房源
正在爬取第 19 页...
第 19 页爬取完成，共找到 60 个房源
正在爬取第 20 页...
第 20 页爬取完成，共找到 60 个房源
浏览器已关闭

爬取到的数据前5行:
  面积(㎡) 总价(万) 均价(元/㎡)
0    49    15    3061
1   110   110   10000
2   118    45    3813
3    78    55    7051
4     9   150    9560

数据已保存到 '房价数据.xlsx'，共 1200 条记录

数据统计信息:
       面积(㎡) 总价(万) 均价(元/㎡)
count   1200  1200    1200
unique   201   302     830
top       78    